In [1]:
import numpy as np
from scipy.spatial import ConvexHull

In [2]:
def writeVTKSurface(outFileName,x,y,z,triangles):
    nodesNumber=x.size
    trianglesNumber=int(triangles.size/3)
    m=open(outFileName,'w')
    m.write('# vtk DataFile Version 2.0\n')
    m.write('Moebius surface\n')
    m.write('ASCII\n')
    m.write('DATASET POLYDATA\n')
    m.write('POINTS '+str(nodesNumber)+' float\n')
    for node in np.arange(nodesNumber):
        m.write(str(x[node])+' '+str(y[node])+' '+str(z[node])+' \n')

    m.write('POLYGONS '+str(trianglesNumber)+' '+str(trianglesNumber*4)+'\n')
    for triangle in np.arange(trianglesNumber):
        m.write('3 '+str(triangles[triangle,0])+' '+str(triangles[triangle,1])+' '+str(triangles[triangle,2])+' \n')

    m.close()
    return

In [3]:
def writeVTKSurfaceField(outFileName,x,y,z,triangles,field):
    nodesNumber=x.size
    trianglesNumber=int(triangles.size/3)
    m=open(outFileName,'w')
    m.write('# vtk DataFile Version 2.0\n')
    m.write('Moebius surface\n')
    m.write('ASCII\n')
    m.write('DATASET POLYDATA\n')
    m.write('POINTS '+str(nodesNumber)+' float\n')
    for node in np.arange(nodesNumber):
        m.write(str(x[node])+' '+str(y[node])+' '+str(z[node])+' \n')

    m.write('POLYGONS '+str(trianglesNumber)+' '+str(trianglesNumber*4)+'\n')
    for triangle in np.arange(trianglesNumber):
        m.write('3 '+str(triangles[triangle,0])+' '+str(triangles[triangle,1])+' '+str(triangles[triangle,2])+' \n')

    a=field.shape
    m.write('CELL_DATA '+str(trianglesNumber)+'\n')
    m.write('FIELD FieldData 1\n')
    m.write('FIEfaceAttributes '+str(a[1])+' '+str(trianglesNumber)+' float\n')
    for triangle in np.arange(trianglesNumber):
        for i in np.arange(a[1]): m.write(str(field[triangle,i])+' ')
        m.write('\n')

    m.close()
    return

In [4]:
def writeVTKDatasetStructure(outFileName,xNum, yNum, zNum, xMin, yMin, zMin, xSpacing, ySpacing, zSpacing, dataInteriors):
    outFile=open(outFileName,'w')
    outFile.write('# vtk DataFile Version 2.0\n')
    outFile.write('corrected flow of stream with boundary elements\n')
    outFile.write('\n')
    outFile.write('ASCII\n')
    outFile.write('DATASET STRUCTURED_POINTS\n')
    outFile.write('DIMENSIONS '+str(xNum)+' '+str(yNum)+' '+str(zNum)+'\n')
    outFile.write('ORIGIN '+str(xMin)+' '+str(yMin)+' '+str(zMin)+'\n')
    outFile.write('SPACING '+str(xSpacing)+' '+str(ySpacing)+' '+str(zSpacing)+'\n')
    outFile.write('\n')
    outFile.write('POINT_DATA '+str(xNum*yNum*zNum)+'\n')
    outFile.write('SCALARS scalar_name float 1\n')
    outFile.write('LOOKUP_TABLE default\n')
        
    numberOfPoints = xNum*yNum*zNum
    for i in np.arange(numberOfPoints):
        outFile.write(str(dataInteriors[i])+'\n')
    outFile.close()
    return

In [5]:
# mesh of one sphere
def fibonacci_covering(samples):

    increment = np.pi * ( 3 - 5**0.5 ) #fibonacci number
    points = np.zeros((samples,3))

    phi = np.arange(samples) * increment
    points[:,1] = ( 2. * np.arange(samples) + 1 )/samples - 1.
    r = ( 1 - points[:,1]**2 )**0.5      
    points[:,0] = np.cos(phi) * r
    points[:,2] = np.sin(phi) * r
      
    return(points,ConvexHull(points))

In [6]:
# Sphere packing
rMin = np.float64(0.03)
rMax = np.float64(0.2)
packingTarget = 0.50

In [7]:
#first sphere:
centers = np.random.random((1,3))*(1.0-2*rMin)+rMin
radii = np.array([rMin])
packing=4/3*radii[0]**3*np.pi
attempt=0

In [8]:
while packing<packingTarget:
    attempt+=1
    #rMin=(1./attempt)**0.333
    #rMax=np.min([rMin*100,0.3])
    newCenter = np.random.rand(3)*(1.0-2*rMin)+rMin
    
    #check that it does not exit the box boundaries    
    maxRadius = np.min(np.concatenate(([rMax],newCenter[0:3],1.0-newCenter[0:3])))

    # expand the sphere up to touching the wall or another sphere
    distances = np.linalg.norm(newCenter-centers,axis=1)
    newRadius = np.min([np.min(distances - radii),maxRadius])
    if newRadius>rMin:
        centers = np.append(centers,np.array([newCenter]),axis=0)
        radii = np.append(radii,newRadius)
        packing+=4/3*newRadius**3*np.pi
        print('sphere',radii.size,attempt,rMin,rMax,newCenter,newRadius,packing) 
nSpheres = radii.size

sphere 2 1 0.03 0.2 [0.71541432 0.42009664 0.88159734] 0.11840266417471357 0.007066110200239988
sphere 3 2 0.03 0.2 [0.26378454 0.29763308 0.67902729] 0.2 0.040576431838531116
sphere 4 3 0.03 0.2 [0.11405227 0.7291842  0.07235609] 0.0723560886993407 0.04216320138466712
sphere 5 4 0.03 0.2 [0.71130558 0.10198129 0.22046809] 0.10198128567961114 0.046605932789212774
sphere 6 5 0.03 0.2 [0.24375322 0.9259073  0.03195898] 0.03195898259567304 0.04674266393243626
sphere 7 6 0.03 0.2 [0.29138999 0.31590611 0.03926203] 0.03926203430985162 0.04699618088400342
sphere 8 7 0.03 0.2 [0.91392136 0.08595781 0.74550089] 0.08595781211523276 0.04965656697063569
sphere 9 8 0.03 0.2 [0.5431899  0.58019373 0.73396536] 0.15924374291825943 0.06657171254739983
sphere 10 9 0.03 0.2 [0.66700084 0.21217565 0.14508733] 0.038688425594536016 0.06681427958223626
sphere 11 10 0.03 0.2 [0.38022385 0.1968933  0.39591409] 0.12227270578012023 0.07447160630260537
sphere 12 12 0.03 0.2 [0.45497411 0.59993818 0.23080121] 0.2

sphere 182 392 0.03 0.2 [0.0993407  0.36391031 0.13648881] 0.030794839515605993 0.3150141516006996
sphere 183 394 0.03 0.2 [0.58917332 0.06889539 0.16263524] 0.037143131222317397 0.3152287982667033
sphere 184 401 0.03 0.2 [0.7697152  0.61553768 0.18692124] 0.03337536977769892 0.315384526324643
sphere 185 402 0.03 0.2 [0.94321528 0.74516529 0.15622057] 0.03741327173621045 0.31560389048499715
sphere 186 404 0.03 0.2 [0.47026873 0.28473059 0.51917967] 0.053846208140544444 0.3162578547027448
sphere 187 405 0.03 0.2 [0.94706961 0.80826151 0.83347978] 0.05293039494468932 0.316879015463088
sphere 188 406 0.03 0.2 [0.95066504 0.89873776 0.64526347] 0.049334958766290016 0.31738199801269856
sphere 189 411 0.03 0.2 [0.76574593 0.81591409 0.31237627] 0.03457187699799885 0.3175550822372978
sphere 190 412 0.03 0.2 [0.03183662 0.48097917 0.17890086] 0.0318366223411904 0.31769024889230385
sphere 191 421 0.03 0.2 [0.1559145  0.88514331 0.64705365] 0.04782585087677138 0.31814847172461036
sphere 192 425 

sphere 307 996 0.03 0.2 [0.1144007  0.33939991 0.29123683] 0.04887836973917166 0.36694687781426816
sphere 308 1002 0.03 0.2 [0.96002957 0.90613474 0.78604648] 0.039970427230071826 0.3672143662310431
sphere 309 1008 0.03 0.2 [0.81695735 0.96477228 0.87206442] 0.03040327998869867 0.3673320861423794
sphere 310 1020 0.03 0.2 [0.56300218 0.70635595 0.45281019] 0.034846669948096154 0.36750933051482726
sphere 311 1023 0.03 0.2 [0.11330858 0.12715806 0.06703486] 0.03648691967855967 0.36771280043336185
sphere 312 1026 0.03 0.2 [0.88204274 0.91488841 0.19569443] 0.031708565203089595 0.36784634258886084
sphere 313 1036 0.03 0.2 [0.94777919 0.59049702 0.69329605] 0.05222080704855436 0.36844285481220906
sphere 314 1042 0.03 0.2 [0.3668348  0.27367274 0.89753571] 0.03940853183542389 0.36869922019157164
sphere 315 1050 0.03 0.2 [0.64482892 0.06226285 0.10258268] 0.0390636350194946 0.3689489133081252
sphere 316 1051 0.03 0.2 [0.0633699  0.54823539 0.62146891] 0.03653174690687373 0.3691531340877681
sph

sphere 395 1678 0.03 0.2 [0.94611897 0.22219877 0.79214481] 0.04702604581327549 0.3909154096834587
sphere 396 1683 0.03 0.2 [0.09595042 0.74044328 0.64002001] 0.03422006593549817 0.3910832634671282
sphere 397 1695 0.03 0.2 [0.25434469 0.88351617 0.85978809] 0.03399972665701605 0.39124789570659085
sphere 398 1711 0.03 0.2 [0.03022211 0.42474085 0.14582153] 0.03022211321828614 0.39136352372792343
sphere 399 1718 0.03 0.2 [0.18753129 0.23023235 0.21528885] 0.047116222489911236 0.3918016507110157
sphere 400 1726 0.03 0.2 [0.23095131 0.16450904 0.47761649] 0.038610080478949024 0.3920427471133904
sphere 401 1741 0.03 0.2 [0.84485396 0.86309631 0.33821928] 0.03399997374985081 0.3922073829422709
sphere 402 1750 0.03 0.2 [0.95373494 0.43686898 0.50816363] 0.03538923751211388 0.39239303605104253
sphere 403 1751 0.03 0.2 [0.87457818 0.54275986 0.50565036] 0.03977867565269802 0.3926566932109761
sphere 404 1755 0.03 0.2 [0.66032315 0.95170934 0.13615916] 0.03202091109479764 0.39279422074748793
sphe

sphere 532 3311 0.03 0.2 [0.95560989 0.50388144 0.22716203] 0.04056186799628543 0.4230887465756697
sphere 533 3379 0.03 0.2 [0.48096637 0.21962154 0.83455697] 0.03697025962993558 0.4233004101427175
sphere 534 3382 0.03 0.2 [0.96804858 0.6347291  0.24339249] 0.03195142319951094 0.42343704428406564
sphere 535 3383 0.03 0.2 [0.5342264  0.30252758 0.03824204] 0.038242036475129214 0.42367131156855947
sphere 536 3384 0.03 0.2 [0.86603627 0.33189727 0.76330467] 0.038837610045569154 0.42391669548842925
sphere 537 3390 0.03 0.2 [0.20010098 0.49265715 0.32999875] 0.03478895893305668 0.42409306069324776
sphere 538 3401 0.03 0.2 [0.48235282 0.21336135 0.73965623] 0.030658596932236562 0.42421377130106297
sphere 539 3457 0.03 0.2 [0.55959712 0.0307615  0.66976906] 0.030734659734534278 0.42433538257485987
sphere 540 3463 0.03 0.2 [0.85294334 0.55758045 0.58548217] 0.03549882663274964 0.42452276575450165
sphere 541 3504 0.03 0.2 [0.89253007 0.47316007 0.90315646] 0.036443126409275675 0.424725503910336

sphere 637 5692 0.03 0.2 [0.10708282 0.85361047 0.25750482] 0.03233625121220443 0.4429345805810953
sphere 638 5725 0.03 0.2 [0.49285619 0.09703049 0.67452626] 0.03450139720429178 0.4431066083902706
sphere 639 5735 0.03 0.2 [0.94383546 0.6826414  0.66806424] 0.03191060240777435 0.4432427195134993
sphere 640 5740 0.03 0.2 [0.07489332 0.40552022 0.03993245] 0.030715230827097262 0.4433641003034493
sphere 641 5755 0.03 0.2 [0.86740199 0.11735129 0.85104865] 0.03358269147211153 0.4435227484884623
sphere 642 5764 0.03 0.2 [0.8873768  0.56041097 0.22882558] 0.039937347615055824 0.44378957333267766
sphere 643 5887 0.03 0.2 [0.45086159 0.12011377 0.61463251] 0.031565948634634244 0.44392132167080844
sphere 644 5906 0.03 0.2 [0.92250419 0.37780882 0.04383333] 0.030236599025830695 0.4440371160373807
sphere 645 5967 0.03 0.2 [0.28037949 0.35551027 0.16414504] 0.03427120173274048 0.4442057234288652
sphere 646 5991 0.03 0.2 [0.903579   0.63336114 0.60751611] 0.030555203151227672 0.44432521688869464
sp

sphere 726 10009 0.03 0.2 [0.41509185 0.96604433 0.51172658] 0.03332538861101032 0.4581805414283088
sphere 727 10010 0.03 0.2 [0.04645758 0.89381639 0.96236659] 0.03763341103030804 0.45840380062656666
sphere 728 10041 0.03 0.2 [0.73067067 0.96928834 0.87509234] 0.030711659781661904 0.4585251390851477
sphere 729 10046 0.03 0.2 [0.12173608 0.92337009 0.03781547] 0.03781547215992545 0.4587516542023146
sphere 730 10070 0.03 0.2 [0.9672411  0.77551045 0.92486972] 0.032758902759138064 0.45889891144014316
sphere 731 10091 0.03 0.2 [0.80884681 0.82126306 0.44428291] 0.0317858429636763 0.4590334323542045
sphere 732 10255 0.03 0.2 [0.43293474 0.12470978 0.53195427] 0.03276673797199442 0.45918079527943617
sphere 733 10267 0.03 0.2 [0.80831675 0.95645697 0.12678217] 0.030549211606319474 0.4593002184589229
sphere 734 10277 0.03 0.2 [0.31719023 0.56312434 0.03907314] 0.03895462254858795 0.4595478269942015
sphere 735 10314 0.03 0.2 [0.44045619 0.9691785  0.77080814] 0.030821502161016867 0.45967047204

sphere 816 16035 0.03 0.2 [0.54894742 0.66092371 0.96422869] 0.03103320953310386 0.4725228958446369
sphere 817 16332 0.03 0.2 [0.62471124 0.07201354 0.03166804] 0.03166803760016394 0.47265592660244704
sphere 818 16343 0.03 0.2 [0.60896354 0.79389707 0.20479616] 0.031694728378079545 0.47278929401085085
sphere 819 16702 0.03 0.2 [0.9334068  0.82431424 0.62269711] 0.030326505153286276 0.4729061243678379
sphere 820 16829 0.03 0.2 [0.68728318 0.8746072  0.26903916] 0.03959474548622796 0.4731661410883119
sphere 821 16918 0.03 0.2 [0.81040209 0.2005434  0.58437417] 0.03683194925878866 0.4733754379518838
sphere 822 16950 0.03 0.2 [0.14286681 0.20271324 0.28383886] 0.03593418604244372 0.4735698002588116
sphere 823 17243 0.03 0.2 [0.8813242  0.29034551 0.19451607] 0.03219020480296937 0.4737095206615134
sphere 824 17273 0.03 0.2 [0.82791582 0.90637784 0.46939905] 0.03208390918459417 0.473847861512998
sphere 825 17319 0.03 0.2 [0.51553832 0.04722021 0.22235511] 0.03310737245753044 0.47399986822087

sphere 900 26451 0.03 0.2 [0.96974705 0.40498646 0.10053136] 0.030252953166680085 0.48488915139209515
sphere 901 26706 0.03 0.2 [0.18550177 0.36190726 0.37523997] 0.031222824274261096 0.4850166499180216
sphere 902 26802 0.03 0.2 [0.03194755 0.65125352 0.85932342] 0.030718752674464153 0.4851380724657857
sphere 903 27079 0.03 0.2 [0.62427424 0.16428846 0.5117863 ] 0.03006599167778952 0.4852519177925705
sphere 904 27252 0.03 0.2 [0.94326177 0.81019865 0.03377212] 0.03348090465610061 0.48540912779041023
sphere 905 27767 0.03 0.2 [0.28071978 0.5383805  0.68482616] 0.03510832367200285 0.48559039485003097
sphere 906 28141 0.03 0.2 [0.03262235 0.30142488 0.34332906] 0.03262235207549916 0.4857358182924298
sphere 907 28224 0.03 0.2 [0.95808484 0.1440478  0.16211211] 0.032705040820850476 0.48588235036619065
sphere 908 28232 0.03 0.2 [0.58614032 0.38823355 0.64197237] 0.030631199732819678 0.48600273765410756
sphere 909 28263 0.03 0.2 [0.95715288 0.92136229 0.4571039 ] 0.04255925223836245 0.4863256

sphere 987 51388 0.03 0.2 [0.6400339  0.53460576 0.56886409] 0.03040877300086163 0.497407727890514
sphere 988 51685 0.03 0.2 [0.19966613 0.25373021 0.37410405] 0.030123815120560567 0.49752223132936063
sphere 989 52777 0.03 0.2 [0.66156566 0.03680485 0.81756431] 0.03056815445861344 0.49764187680119804
sphere 990 52837 0.03 0.2 [0.65247025 0.96179762 0.03143241] 0.031094174951124265 0.4977678057919244
sphere 991 53296 0.03 0.2 [0.49916068 0.28551882 0.66198608] 0.031592368403486615 0.4978998852154397
sphere 992 53966 0.03 0.2 [0.55636531 0.21961221 0.68933996] 0.03033747237951573 0.4980168423692707
sphere 993 53979 0.03 0.2 [0.46865086 0.03007535 0.47360922] 0.030075353117909532 0.49813079407086847
sphere 994 54663 0.03 0.2 [0.0362105  0.66961582 0.73504939] 0.030246812177665096 0.49824670581422825
sphere 995 54883 0.03 0.2 [0.69876266 0.66208376 0.21200967] 0.03019802310182254 0.4983620575538117
sphere 996 54897 0.03 0.2 [0.03901249 0.12938289 0.4708146 ] 0.035486296490973235 0.49854924

In [9]:
# write a VTK file with the surfaces
nPoints=500
(points,oneSphereMesh) = fibonacci_covering(nPoints)        
allPoints = np.zeros((0,3),dtype=np.float64)
connectivity = np.zeros((0,3),dtype=np.int64)
for thisSphere in np.arange(nSpheres):
    allPoints=np.append(allPoints, points*radii[thisSphere]+centers[thisSphere], axis=0)
    connectivity=np.append(connectivity, oneSphereMesh.simplices+thisSphere*nPoints,axis=0)

writeVTKSurface('sphere_packing.vtk',allPoints[:, 0], allPoints[:, 1], allPoints[:, 2],connectivity)

In [10]:
# extract a 2D section in the mid of the packing
xSize=1001;ySize=1001;zSize=1;

# Create an array for the calculations
matrix = np.zeros((xSize,ySize,zSize))
X=np.arange(xSize)*(1/xSize)
Y=np.arange(ySize)*(1/ySize)
Z=0.5
coordinates=np.meshgrid(X,Y,Z)
for i in np.arange(nSpheres):
    if (centers[i,2]+radii[i]>0.5) and (centers[i,2]-radii[i]<0.5):
        #interiors = np.linalg.norm(coordinates-centers[i])<(radii[i])
        interiors = (coordinates[0]-centers[i,0])**2+(coordinates[1]-centers[i,1])**2+(coordinates[2]-centers[i,2])**2<radii[i]**2
        matrix[interiors]=1
        print('calculating the sphere',i)

writeVTKDatasetStructure('slice_packing.vtk',xSize,ySize,zSize, 0.0,0.0,0.0, 1/(xSize-1), 1/(ySize-1), 0.01, matrix.reshape(xSize*ySize*zSize) )

calculating the sphere 2
calculating the sphere 10
calculating the sphere 25
calculating the sphere 31
calculating the sphere 41
calculating the sphere 46
calculating the sphere 50
calculating the sphere 58
calculating the sphere 61
calculating the sphere 68
calculating the sphere 76
calculating the sphere 94
calculating the sphere 95
calculating the sphere 109
calculating the sphere 133
calculating the sphere 139
calculating the sphere 146
calculating the sphere 147
calculating the sphere 154
calculating the sphere 161
calculating the sphere 175
calculating the sphere 176
calculating the sphere 180
calculating the sphere 185
calculating the sphere 191
calculating the sphere 196
calculating the sphere 197
calculating the sphere 214
calculating the sphere 233
calculating the sphere 236
calculating the sphere 239
calculating the sphere 241
calculating the sphere 244
calculating the sphere 260
calculating the sphere 262
calculating the sphere 268
calculating the sphere 277
calculating the

In [11]:
if 0:
    # write a VTK file with 3D volume file description
    xSize=301;ySize=301;zSize=301;

    # Create an array for the calculations
    matrix = np.zeros((xSize,ySize,zSize))
    X=np.arange(xSize)*(1/xSize)
    Y=np.arange(ySize)*(1/ySize)
    Z=np.arange(zSize)*(1/zSize)
    coordinates=np.meshgrid(X,Y,Z)
    for i in np.arange(nSpheres):
        interiors = np.linalg.norm(coordinates-centers[i])<(radii[i])
        #interiors = (coordinates[0]-centers[i,0])**2+(coordinates[1]-centers[i,1])**2+(coordinates[2]-centers[i,2])**2<radii[i]**2
        matrix[interiors]=1
        print('calculating the sphere',i)

    writeVTKDatasetStructure('interior_packing.vtk',xSize,xSize,xSize, 0.0,0.0,0.0, 1/(xSize-1), 1/(ySize-1), 1/(zSize-1), matrix.reshape(101*101*101) )


            
            
        
        
        
        
        
        
        
        
        
        
        